In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np


from keras.utils import image_dataset_from_directory
from keras.models import Sequential
from keras.layers import Rescaling, Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Activation
from keras.metrics import Precision, Recall

In [2]:
# get the data directories
dir = './archive'
train_dir = dir + '/train'
test_dir = dir + '/test'

In [3]:
# generate tf datasets

train = image_dataset_from_directory(
    train_dir,
    label_mode = 'binary',
    image_size = (32, 32)
)

test = image_dataset_from_directory(
    test_dir,
    label_mode = 'binary',
    image_size = (32, 32)
)

Found 100000 files belonging to 2 classes.
Found 20000 files belonging to 2 classes.


In [5]:
# create a CNN Sequential model using Keras
model = Sequential([
    Rescaling(scale = 1./255),
    Conv2D(filters = 64, kernel_size = 3, activation = 'relu'),
    MaxPooling2D(),
    Conv2D(filters = 64, kernel_size = 3, activation = 'relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(units = 64),
    BatchNormalization(),
    Activation('relu'),
    Dense(units = 1, activation = 'sigmoid')
])

# compile the model with an optimizer, loss function, and metrics
model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy', Precision(), Recall()]
)

# train the model
model.fit(train, validation_data = test, epochs = 5)

Epoch 1/5
3125/3125 [==============================] - 78s 24ms/step - loss: 0.2792 - accuracy: 0.8821 - precision_1: 0.8786 - recall_1: 0.8867 - val_loss: 0.2554 - val_accuracy: 0.8899 - val_precision_1: 0.9013 - val_recall_1: 0.8758
Epoch 2/5
3125/3125 [==============================] - 48s 15ms/step - loss: 0.2064 - accuracy: 0.9168 - precision_1: 0.9163 - recall_1: 0.9174 - val_loss: 0.3044 - val_accuracy: 0.8751 - val_precision_1: 0.9034 - val_recall_1: 0.8399
Epoch 3/5
3125/3125 [==============================] - 47s 15ms/step - loss: 0.1825 - accuracy: 0.9262 - precision_1: 0.9254 - recall_1: 0.9271 - val_loss: 0.2846 - val_accuracy: 0.8716 - val_precision_1: 0.9800 - val_recall_1: 0.7588
Epoch 4/5
3125/3125 [==============================] - 47s 15ms/step - loss: 0.1628 - accuracy: 0.9361 - precision_1: 0.9351 - recall_1: 0.9373 - val_loss: 0.1892 - val_accuracy: 0.9253 - val_precision_1: 0.9255 - val_recall_1: 0.9251
Epoch 5/5
3125/3125 [==============================] - 46s 1

In [ ]:
# evaluate the model on the test set
y_pred = model.predict(test)

625/625 [==============================] - 10s 15ms/step


In [ ]:
from keras.metrics import Precision, Recall, BinaryAccuracy

#calculate precision, recall, and accuracy
pre = Precision()
rec = Recall()
acc = BinaryAccuracy()

for batch in test.as_numpy_iterator(): 
    X, y = batch
    yhat = model.predict(X)
    pre.update_state(y, yhat)
    rec.update_state(y, yhat)
    acc.update_state(y, yhat)
    
print(f'Recall: {rec.result().numpy()}, Precision: {pre.result().numpy()}, Accuracy: {acc.result().numpy()}')

1/1 [==============================] - 0s 40ms/step
Recall: 0.9735000133514404, Precision: 0.8351204991340637, Accuracy: 0.8906499743461609


In [10]:
import cv2

In [8]:
# function to predict whether image at the given path is real or fake
def predict_image(image_path):
    # load the image and resize it to 32x32
    img = cv2.imread(image_path)
    img = tf.image.resize(img, (32, 32))
    
    # predict the class
    y_prob = model.predict(np.expand_dims(img, 0))
    return 'REAL' if y_prob[0]>0.5 else 'FAKE'

In [ ]:
import pickle

In [ ]:
# save the model to a file
model_pkl_file = "model.pkl"  

with open(model_pkl_file, 'wb') as file:  
    pickle.dump(model, file)

In [6]:
model.save('./model.keras')

In [17]:
image_path = './Test_Image.jpeg'

predict_image(image_path)

1/1 [==============================] - 0s 43ms/step


'REAL'